# Solve the Robertson Equations
Solve the Robertson Equations using Explicit methods
These equations are a stiff system, and so explicit methods can have trouble integrating.

Robertson Equations:
$
\frac{dx}{dt} = -0.04x + 10000yz \\
\frac{dy}{dt} = 0.04x - 10000yz - 30000000y^2 \\
\frac{dz}{dt} = 30000000y^2
$

In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), ".."))
import rungekutta as rk
import numpy as np
import pandas as pd
from plotnine import *
import time

In [ ]:
def robertson_rhs(y, t):
    return np.ndarray([
        -0.04 * y[0] + 10000 * y[1] * y[2],
        0.04 * y[0] - 10000 * y[1] * y[2] - 30000000 * y[1] * y[1],
        30000000 * y[1] * y[1]
    ], dtype=float)